## Import of relevant python modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import plotly.offline
import plotly.graph_objects as go
from scipy.stats import ks_2samp
from sbemdb import SBEMDB
from cleandb import clean_db, clean_db_uct
import os
import scipy.io
import math
import cmath
import pickle
import random
import csv
from mapping import Mapping

mapping = Mapping()
np.warnings.filterwarnings('ignore')

In [2]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Import database

In [3]:
db = SBEMDB() # connect to DB
db = clean_db(db)
x,y,z = db.segments(444)

## Load phase and magnitude values

In [38]:
#Get mag and phase values for each roi
col_names =  ['roi','mag','phase','coherence']
data_coh = pd.DataFrame(columns = col_names)

file_name = '83'

path_mag = os.path.join('..', 'data', 'roi_mag_' + file_name + '.mat')
mat = scipy.io.loadmat(path_mag)

path_phase = os.path.join('..', 'data', 'roi_phase_' + file_name + '.mat')
phase_mat = scipy.io.loadmat(path_phase)

#save magnitude values in mag column
for roin, mag in enumerate(mat['roi_mag'][0]):
    data_coh.loc[roin,'roi'] = roin + 1
    data_coh.loc[data_coh['roi'] == roin + 1, 'mag'] = mag 
    
#save phase values in phase column
# phase is between -pi and pi 
for roin, phase in enumerate(phase_mat['roi_phase'][0]):
    data_coh.loc[roin,'roi'] = roin + 1
    data_coh.loc[data_coh['roi'] == roin + 1, 'phase'] = phase

#Calculate Coherence:
for i,val in data_coh.iterrows():
    data_coh.loc[i,'coherence'] = val['mag'] * math.cos(val['phase']) +  1j*val['mag']*math.sin(val['phase']);

## Cluster-based ANOVA analysis

In [39]:
#load data
#then construct a dataframe with columns'sid','tid','cluster_id','coherence'
#note that synapses which are "unclustered" are excluded from the following analysis

#load data from .pkl file
data_pkl = load_obj("saved_param_clusters_path_2-5"); 

In [40]:
#Dataframe with columns'sid','tid','cluster_id','coherence'
#Note that synapses which are "unclustered" are excluded from the following analysis

#prepare all needed variables
cluster_id = 0;
data_by_params = dict();
corresponding_tid = dict();

col_names =  ['sid','tid','cluster_id','coherence'];

'''Method for getting the corresponding tid
We are storing already recovered tids since it's faster to get them from dictionary than 
to execute query each time we need the tid.
'''
def find_corresponding_tid(sid):
    global corresponding_tid;
    if sid in corresponding_tid.keys():
        return corresponding_tid[sid];
    try:
        tid = db.synapses(f's.sid={sid}')[3][0];
        corresponding_tid[sid] = tid;
        return tid;
    except:
        return np.NaN;
    
    
for key in data_pkl:
    counter = 0;
    dframe = pd.DataFrame(columns = col_names);
    for synapse_group in data_pkl[key]['clusters']:
        for synapse in synapse_group:
            dframe.loc[counter,'cluster_id'] = cluster_id;
            dframe.loc[counter,'sid'] = synapse.sid;
            
            tid = find_corresponding_tid(synapse.sid);
            roi = mapping.sbem2roi[tid];
            
            dframe.loc[counter,'tid'] = tid;
            dframe.loc[counter,'coherence'] = data_coh[data_coh['roi']==roi]['coherence'].iloc[0];
            counter += 1;
        cluster_id += 1;
    data_by_params[key] = dframe



In [41]:
def anova(data, randomize_cluster=False):
    data = data.copy()
    if randomize_cluster:
        
        tid_coh_df = data.groupby('tid').first()[['coherence']]
        coherences = tid_coh_df['coherence'].values
        np.random.shuffle(coherences)
        for i, tid in enumerate(tid_coh_df.index):
            data.loc[data['tid'] == tid, 'coherence'] = coherences[i]
        
               
    clusters = np.unique(data['cluster_id'])
    clusters_used = {cluster_id: True for cluster_id in clusters}
    
    n_clust = 0 # number of clusters
    n_tot = 0   # total number of synapses used in the analysis
    # Calculate the mean of coherence values X_j for each cluster j
    means_per_cluster = []
    for cluster_id in clusters:
        clust_data_grouped = data[data['cluster_id'] == cluster_id].groupby('tid').first()
        if len(clust_data_grouped) > 1:
            means_per_cluster.append(clust_data_grouped['coherence'].mean())
            n_clust += 1
            n_tot += len(clust_data_grouped)
        else:
            clusters_used[cluster_id] = False
    
    # Calculating degrees of freedom
    # Variable df_clust: k-1, with k the number of clusters 
    # Variable df_tot: N-1, with N the number of total synapses
    # Variable df_res: N  - k 
    df_clust = n_clust - 1
    df_tot = n_tot - 1 
    df_res = n_tot - n_clust 
    

    # 2. Calculate the mean of coherence values X_t of all synapses
    mean_all_synapses = data[data['cluster_id'].isin([cl_id for cl_id in clusters_used.keys() if clusters_used[cl_id]])]\
                        .groupby(['cluster_id', 'tid']).first()['coherence'].mean()
    
    # Calculation of sums of absolute squared differences

    # SSB: Sum_j(abs((X_j-X_t))^2)
    # X_j-X_t: Subtract the mean of each cluster from the total mean. Square and take the absolute value, and sum all terms
    # up
    SSB = 0
    i = 0
    for cluster_id in clusters:
        if clusters_used[cluster_id]:
            n = np.unique(data.loc[data['cluster_id'] == cluster_id, 'tid']).shape[0]
            SSB += n*(np.square(abs(means_per_cluster[i] - mean_all_synapses)))
            i += 1
        

    # SSE: Sum_j Sum_i (abs(X_i-X_j)^2); X_i are coherence values for single synapses in the cluster j
    # X_i-X_j: Subtract each synapse's coherence from its cluster's mean value. Summation for all synapses in a cluster
    # and then over all clusters
    SSE = 0
    i = 0
    for cluster_id in clusters:
        if clusters_used[cluster_id]:
            cluster_mean = means_per_cluster[i]
            clust_data_grouped = data[data['cluster_id'] == cluster_id].groupby('tid').first()
            for j, row in clust_data_grouped.iterrows():
                SSE += np.square(abs(row['coherence'] - cluster_mean))
            i += 1
        
    # Calculate MSE and MSB   
    MSB = SSB/df_clust
    MSE = SSE/df_res
    
    #F-ratio
    F = MSB/MSE
    
    # Check p-value: 1-cdf of F-distribution
    p_value = 1-scipy.stats.f.cdf(F, df_clust, df_res)
    return F, p_value

## Define empirical p-values

In [42]:
#Empirical p-value calculation
#Davison & Hickley, 1997

def test_rand(F_true, x_times, data):
    n_times_larger = 0
    for i in range(x_times):
        F_rand, _ = anova(data, randomize_cluster=True)
        if F_true < F_rand: 
            n_times_larger += 1
    p_value = (n_times_larger+1)/(x_times+1)
    return p_value

In [43]:
for val in data_by_params:
    if val[1]>105: continue
    if val[0]>val[1]: continue
    F_true, p_value = anova(data_by_params[val])
    print(f'Results parametrical: {val} | F true={F_true} | p_value={p_value}')
    

Results parametrical: (2.5, 10) | F true=2.4370629073223147 | p_value=0.15800447380484473
Results parametrical: (2.5, 15) | F true=2.6645209891121637 | p_value=0.12208284830835237
Results parametrical: (2.5, 20) | F true=1.464048413936139 | p_value=0.3004284698482521
Results parametrical: (2.5, 25) | F true=1.3802808501078419 | p_value=0.32271160400400833
Results parametrical: (2.5, 30) | F true=1.0829715717003345 | p_value=0.430848235193356
Results parametrical: (2.5, 35) | F true=1.0829715717003345 | p_value=0.430848235193356
Results parametrical: (2.5, 40) | F true=1.0829715717003345 | p_value=0.430848235193356
Results parametrical: (2.5, 45) | F true=0.698844641589565 | p_value=0.6737433974746359
Results parametrical: (2.5, 50) | F true=1.1702140340850742 | p_value=0.3790550983858687
Results parametrical: (2.5, 55) | F true=1.1702140340850742 | p_value=0.3790550983858687
Results parametrical: (2.5, 60) | F true=1.406184559587339 | p_value=0.27954658287140366
Results parametrical: (

Results parametrical: (12.5, 75) | F true=0.8210006657371824 | p_value=0.6733201954842469
Results parametrical: (12.5, 80) | F true=0.8358646471227621 | p_value=0.6518817943685307
Results parametrical: (12.5, 85) | F true=0.8543253122830409 | p_value=0.6212837683409135
Results parametrical: (12.5, 90) | F true=0.8543253122830409 | p_value=0.6212837683409135
Results parametrical: (12.5, 95) | F true=0.9139887843544612 | p_value=0.5544394486775837
Results parametrical: (12.5, 100) | F true=0.9139887843544612 | p_value=0.5544394486775837
Results parametrical: (12.5, 105) | F true=0.9966914986197339 | p_value=0.4672749129190056
Results parametrical: (15.0, 15) | F true=0.7235080684614674 | p_value=0.8220835933593079
Results parametrical: (15.0, 20) | F true=0.725874589294828 | p_value=0.8225445781994669
Results parametrical: (15.0, 25) | F true=0.6619122310919849 | p_value=0.8793109834777736
Results parametrical: (15.0, 30) | F true=0.526351925188217 | p_value=0.9700765858721305
Results pa

Results parametrical: (25.0, 85) | F true=1.0106739108137373 | p_value=0.4629404832666668
Results parametrical: (25.0, 90) | F true=1.0414843909976426 | p_value=0.43084904988574024
Results parametrical: (25.0, 95) | F true=1.1337943052358432 | p_value=0.34679879547930825
Results parametrical: (25.0, 100) | F true=1.0577013298795097 | p_value=0.41346991612504747
Results parametrical: (25.0, 105) | F true=1.1267173774334658 | p_value=0.3545363313799177
Results parametrical: (27.5, 30) | F true=0.5327377924078488 | p_value=0.9746447970175145
Results parametrical: (27.5, 35) | F true=0.7073656439838564 | p_value=0.8607421377014762
Results parametrical: (27.5, 40) | F true=0.6688472577921057 | p_value=0.8978306345703927
Results parametrical: (27.5, 45) | F true=0.6482920413774932 | p_value=0.9119211444479347
Results parametrical: (27.5, 50) | F true=0.6724366315194644 | p_value=0.8822661818883223
Results parametrical: (27.5, 55) | F true=0.6528748158866045 | p_value=0.8961984192709052
Resul

In [10]:
f_dict = {}
for params in data_by_params:
    f_dict[params],_ = anova(data_by_params[params]) 
save_obj(f_dict, 'F_values_by_params_2-5_newdb' + file_name)

In [37]:
def get_num_different_roi(clusters):
    counter = 0
    synapses_counter = 0
    for clust in clusters:
        rois = set()
        for syn in clust:
            tid = find_corresponding_tid(syn.sid)
            roi = mapping.sbem2roi[tid]
            rois.add(roi)
        if(len(rois) == 1): continue;
        counter += 1
        synapses_counter += len(clust)
    return counter,synapses_counter

In [ ]:
#Calculte the empirical p-value and further cluster statistics
col_names =  ['Params','F_value','p_value','Numb_clusters','Numb_clusters_heterogen','Total_syn_clusters','Total_syn_heterogen_clusters']
results_anova = pd.DataFrame(columns = col_names)

for i, val in enumerate(data_by_params):
    if val[1]>105: continue
    if val[0]>val[1]: continue
    F_true, foo = anova(data_by_params[val])
    p_value_rand = test_rand(F_true, 1000, data_by_params[val])
    
    total_syn_clust = sum([len(el) for el in data_pkl[val]['clusters']])
    clust_het,syn_het = get_num_different_roi(data_pkl[val]['clusters'])
    
    results_anova.loc[i,'Params'] = val
    results_anova.loc[i,'F_value'] = F_true
    results_anova.loc[i,'p_value'] = p_value_rand
    results_anova.loc[i,'Numb_clusters'] = len(data_pkl[val]['clusters'])
    results_anova.loc[i,'Numb_clusters_heterogen'] = clust_het
    results_anova.loc[i,'Total_syn_clusters'] = total_syn_clust
    results_anova.loc[i,'Total_syn_heterogen_clusters'] = syn_het
    
print(type(results_anova))
results_anova.to_csv('Results_' + file_name + '.csv')

In [47]:
#Example calculation
data_params = data_by_params[(5, 55)]

# result 1
F_true, p_value = anova(data_params)
print(f'Results parametrical: F true={F_true} | p_value={p_value}')

# result 2 - randomized
p_value_rand = test_rand(F_true, 300, data_params)

print(f'Results randomized: F true={F_true} | p_value={p_value_rand}')

Results parametrical: F true=1.5811543052589698 | p_value=0.13374925112665892
Results randomized: F true=1.5811543052589698 | p_value=0.13861386138613863
